LIBRARIES

In [119]:
import streamlit as st
import pandas as pd
import pymysql
import plotly.express as px
from streamlit_option_menu import option_menu
import requests
import json
from PIL import Image

SQL CONNECTION

In [30]:
#sql connection
myconnection = pymysql.connect(host = '127.0.0.1',user='root',passwd='root',database = "phonepe",port=3306)
cur = myconnection.cursor()

DATA FRAMES

In [31]:
#Agg_TX_data
cur.execute("select * from agg_tx")
myconnection.commit()
F1 = cur.fetchall()

AGG_TX_DF= pd.DataFrame(F1, columns=('States', 'Years', 'Quaters', 'Transaction_type', 'Transaction_count', 'Transaction_amount'))


#Agg_US_data
cur.execute("select * from agg_us")
myconnection.commit()
F2 = cur.fetchall()

AGG_US_DF= pd.DataFrame(F2, columns=('States', 'Years', 'Quaters', 'Brands', 'Transaction_count', 'Percentage'))


#Agg_INS_data
cur.execute("select * from agg_ins")
myconnection.commit()
F3 = cur.fetchall()

AGG_INS_DF= pd.DataFrame(F3, columns=('States', 'Years', 'Quaters', 'Transaction_type', 'Transaction_count', 'Transaction_amount'))

In [32]:
#MAP_TX_data
cur.execute("select * from map_tx")
myconnection.commit()
F4 = cur.fetchall()

MAP_TX_DF= pd.DataFrame(F4, columns=('States', 'Years', 'Quaters', 'Districts', 'Transaction_count', 'Transaction_amount'))


#MAP_US_data
cur.execute("select * from map_user")
myconnection.commit()
F5 = cur.fetchall()

MAP_US_DF= pd.DataFrame(F5, columns=('States', 'Years', 'Quaters', 'Districts', 'Registered_Users', 'App_Opens'))


#MAP_INS_data
cur.execute("select * from map_ins")
myconnection.commit()
F6 = cur.fetchall()

MAP_INS_DF= pd.DataFrame(F6, columns=('States', 'Years', 'Quaters', 'Districts', 'Transaction_count', 'Transaction_amount'))


In [33]:
#TOP_TX_data
cur.execute("select * from top_tx")
myconnection.commit()
F7 = cur.fetchall()

TOP_TX_DF= pd.DataFrame(F7, columns=('States', 'Years', 'Quaters', 'Pincodes', 'Transaction_count', 'Transaction_amount'))


#TOP_US_data
cur.execute("select * from top_user")
myconnection.commit()
F8 = cur.fetchall()

TOP_US_DF= pd.DataFrame(F8, columns=('States', 'Years', 'Quaters', 'Pincodes', 'Registered_Users'))


#MAP_INS_data
cur.execute("select * from top_ins")
myconnection.commit()
F9 = cur.fetchall()

TOP_INS_DF= pd.DataFrame(F9, columns=('States', 'Years', 'Quaters', 'Pincodes', 'Transaction_count', 'Transaction_amount'))

FUNCTIONS

TRANSACTION COUNT AND AMOUNT SORTED BY YEAR

In [34]:
def TX_count_amount(DF, YR):    
    year=DF[DF["Years"]==YR]
    year.reset_index(drop= True, inplace = True)

    group_year=year.groupby("States")[["Transaction_count","Transaction_amount"]].sum()
    group_year.reset_index(inplace = True)

    
    fig_count= px.bar(group_year, x="States", y="Transaction_count", title=f"{YR} TRANSACTION COUNT",
                    color_discrete_sequence=px.colors.sequential.Bluyl_r,height=600, width=600)
    fig_count.show()
   
    fig_amount= px.bar(group_year, x="States", y="Transaction_amount", title=f"{YR} TRANSACTION AMOUNT",
                    color_discrete_sequence=px.colors.sequential.Reds, height=600, width=600)
    fig_amount.show()

    url="https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson"
    data = requests.get(url)
    coordinate=json.loads(data.content)
    state_names=[]
    for i in coordinate["features"]:
        A=i["properties"]["ST_NM"]
        state_names.append(A)

    state_names.sort()

   

    fig_geo_count=px.choropleth(group_year, geojson=coordinate, locations="States",featureidkey="properties.ST_NM",
                            color="Transaction_count",color_continuous_scale="ylgnbu",height=500,width=500,
                            range_color=(group_year["Transaction_count"].min(),group_year["Transaction_count"].max()),
                            hover_name="States", title= f"{YR} TRANSACTION COUNT",fitbounds="locations")
    fig_geo_count.update_geos(visible= False)
    fig_geo_count.show()
    

    fig_geo_amount=px.choropleth(group_year, geojson=coordinate, locations="States",featureidkey="properties.ST_NM",
                            color="Transaction_amount",color_continuous_scale="ylorrd",height=500,width=500,
                            range_color=(group_year["Transaction_amount"].min(),group_year["Transaction_amount"].max()),
                            hover_name="States", title= f"{YR} TRANSACTION AMOUNT",fitbounds="locations")
    fig_geo_amount.update_geos(visible= False)
    fig_geo_amount.show()
    
    return year

SORTED TRANSACTION COUNT AND AMOUNT GROUPED BY QUARTER

In [35]:
def TX_count_amount_Q(DF, Q):    
    year=DF[DF["Quaters"]==Q]
    year.reset_index(drop= True, inplace = True)

    group_year=year.groupby("States")[["Transaction_count","Transaction_amount"]].sum()
    group_year.reset_index(inplace = True)


    fig_count= px.bar(group_year, x="States", y="Transaction_count", title=f"YEAR {Q} QUATER TRANSACTION COUNT",
                        color_discrete_sequence=px.colors.sequential.Bluyl_r,height=600, width=600)
    fig_count.show()



    fig_amount= px.bar(group_year, x="States", y="Transaction_amount", title=f"YEAR {Q} QUATER TRANSACTION AMOUNT",
                    color_discrete_sequence=px.colors.sequential.Reds, height=600, width=600)
    fig_amount.show()

    url="https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson"
    data = requests.get(url)
    coordinate=json.loads(data.content)
    state_names=[]
    for i in coordinate["features"]:
        A=i["properties"]["ST_NM"]
        state_names.append(A)

    state_names.sort()

    fig_geo_count=px.choropleth(group_year, geojson=coordinate, locations="States",featureidkey="properties.ST_NM",
                            color="Transaction_count",color_continuous_scale="Tealgrn",height=600,width=600,
                            range_color=(group_year["Transaction_count"].min(),group_year["Transaction_count"].max()),
                            hover_name="States", title= f"YEAR {Q} QUATER TRANSACTION COUNT",fitbounds="locations")
    fig_geo_count.update_geos(visible= False)
    fig_geo_count.show()



    fig_geo_amount=px.choropleth(group_year, geojson=coordinate, locations="States",featureidkey="properties.ST_NM",
                            color="Transaction_amount",color_continuous_scale="solar",height=500,width=500,
                            range_color=(group_year["Transaction_amount"].min(),group_year["Transaction_amount"].max()),
                            hover_name="States", title= f"YEAR {Q} QUATER TRANSACTION AMOUNT",fitbounds="locations")
    fig_geo_amount.update_geos(visible= False)
    fig_geo_amount.show()
    
    return year


AGGRATED DATA FUNCTIONS

TRANSACTION

In [36]:
def agg_txtype(DF,state):

    year=DF[DF["States"]== state]
    year.reset_index(drop= True, inplace = True)

    group_year=year.groupby("States")[["Transaction_count","Transaction_amount"]].sum()
    group_year.reset_index(inplace = True)


    figpie_count=px.pie(data_frame=group_year,names="Transaction_type", values="Transaction_count",title=f"{state.upper()} TRANSACTION COUNT",
                    color_discrete_sequence=px.colors.sequential.Bluyl_r, hole= 0.1, width=600)
    # fig_count= px.bar(group_year, x="Transaction_type", y="Transaction_count", title="TRANSACTION COUNT",
    #                     color_discrete_sequence=px.colors.sequential.Bluyl_r,height=700, width=600)
    figpie_count.show()


    figpie_amount=px.pie(data_frame=group_year,names="Transaction_type", values="Transaction_amount",title=f"{state.upper()}TRANSACTION AMOUNT",
                    color_discrete_sequence=px.colors.sequential.Agsunset_r,hole= 0.1, width=600)
    # fig_amount= px.bar(group_year, x="Transaction_type", y="Transaction_amount", title="TRANSACTION AMOUNT",
    #                 color_discrete_sequence=px.colors.sequential.Agsunset_r, height=700, width=600)
    figpie_amount.show()

In [ ]:
AGGTX_Y_DATA= TX_count_amount(AGG_TX_DF,YR)

In [ ]:
agg_txtype(AGGTX_Y_DATA,state) 

In [ ]:
AGGTX_Y_Q_DATA=TX_count_amount_Q(AGGTX_Y_DATA,Quat)

In [ ]:
agg_txtype(AGGTX_Y_Q_DATA,state) 

USER

In [ ]:
def aggusbra(DF,YR):

    year=DF[DF["Years"]==YR]
    year.reset_index(drop= True, inplace = True)

    group_year=year.groupby("Brands")[[ "Transaction_count","Percentage"]].sum()
    group_year.reset_index(inplace = True)


    # figpie_count=px.pie(data_frame=group_year,names="Brands", values="Transaction_count",title= "TRANSACTION COUNT",
    #                 color_discrete_sequence=px.colors.sequential.Blackbody, hole= 0.1, width=600)
    fig_count= px.bar(group_year, x="Brands", y="Transaction_count", title=f"{YR} TRANSACTION COUNT",
                        color_discrete_sequence=px.colors.sequential.Blackbody_r, width=800)
    fig_count.show()


    # figpie_amount=px.pie(data_frame=group_year,names="Brands", values="Percentage",title="PERCENTAGE",
    #                 color_discrete_sequence=px.colors.sequential.Rainbow,hole= 0.1, width=800)
    fig_amount= px.bar(group_year, x="Brands", y="Percentage", title=f"{YR} PERCENTAGE",
                    color_discrete_sequence=px.colors.sequential.Rainbow, width=800)
    fig_amount.show()

    return year

In [ ]:
def Agguser_yr_b(DF,state):
    year=DF[DF["States"]==state]
    year.reset_index(drop= True, inplace = True)

    group_year=year.groupby(["Brands", "Quaters"])[[ "Transaction_count"]].sum()
    group_year.reset_index(inplace = True)

    fig_count= px.bar(year, x="Brands", y="Transaction_count", title=f"{year["Years"].min()} YEAR {state.upper()} Transaction Count",
                            color_continuous_scale=px.colors.sequential.tempo_r, height=800,width=1000, hover_name="Quaters")  

    st.plotly_chart(fig_count)
    return year

In [ ]:
AGGUSYEAR=aggusbra(AGG_US_DF,YR)

In [ ]:
AGGUSY_Q_S=Agguser_yr_b(AGGUSYEAR,state)

INSURANCE

In [ ]:
AGGINSDATA= TX_count_amount(AGG_INS_DF,YR)

In [ ]:
TX_count_amount_Q(AGGINSDATA,Quat)

MAP DATA FUNCTIONS

TRANSACTION

In [ ]:
MAPTXDATA_Y= TX_count_amount(MAP_TX_DF,YR)

In [ ]:
Mapins_yr_dis(MAPTXDATA_Y,state)

In [ ]:
MAPTXDATA_Y_Q= TX_count_amount_Q(MAPTXDATA_Y,Quat)

In [ ]:
Mapins_yr_dis(MAPTXDATA_Y_Q,state) 

USER

In [ ]:
def Mapuser(DF,YR):
    year=DF[DF["Years"]==YR]
    year.reset_index(drop= True, inplace = True)

    group_year=year.groupby("States")[[ "Registered_Users","App_Opens"]].sum()
    group_year.reset_index(inplace = True)

    fig_count= px.line(group_year, x="States", y=["Registered_Users","App_Opens"], title=f"{YR} REGISTERED USERS",
                        markers= True)
    fig_count.show()

    return year

In [ ]:
def Mapuser_Q(DF,Q):
    year=DF[DF["Quaters"]==Q]
    year.reset_index(drop= True, inplace = True)

    group_year=year.groupby("States")[[ "Registered_Users","App_Opens"]].sum()
    group_year.reset_index(inplace = True)

    fig_count= px.line(group_year, x="States", y=["Registered_Users","App_Opens"], title=f"{year["Years"].unique()} YEAR {Q} QUATER REGISTERED USERS & APP OPENS",
                        markers= True)
    fig_count.show()

    return year

In [ ]:
def MapuserY_Q_S(DF,State):
    year=DF[DF["States"]==State]
    year.reset_index(drop= True, inplace = True)


    fig_rig= px.bar(year, x="Registered_Users", y="Districts", title=f"{year["Years"].min()} YEAR {year["Quaters"].min()} QUATER {State.upper()} REGISTERED USERS",
                        color_discrete_sequence=px.colors.sequential.tempo_r, height=800)
    fig_rig.show()
    
    fig_apps= px.bar(year, x="App_Opens", y="Districts", title=f"{year["Years"].min()} YEAR {year["Quaters"].min()} QUATER {State.upper()} App_Opens",
                        color_discrete_sequence=px.colors.sequential.Redor, height=800)
    fig_apps.show()
    
    return year

In [ ]:
MAPUSERDATA_Y= Mapuser(MAP_US_DF,YR)

In [ ]:
MAPUSERDATA_Y_Q= Mapuser_Q(MAPUSERDATA_Y, Quat)

In [ ]:
MapuserY_Q_S(MAPUSERDATA_Y_Q,state)

INSURANCE

In [ ]:
def Mapins_yr_dis(DF,Dis):

    year=DF[DF["States"]== Dis]
    year.reset_index(drop= True, inplace = True)

    group_year=year.groupby("Districts")[["Transaction_count","Transaction_amount"]].sum()
    group_year.reset_index(inplace = True)

    fig_count= px.bar(group_year, x="Transaction_count", y="Districts",orientation="h", title=f"{Dis.upper()} DISTRICT & TRANSACTION COUNT",
                            color_discrete_sequence=px.colors.sequential.Mint_r)
    fig_count.show()


    fig_amount= px.bar(group_year, x="Transaction_amount", y="Districts",orientation="h", title=f"{Dis.upper()} DISTRICT & TRANSACTION AMOUNT",
                            color_discrete_sequence=px.colors.sequential.Cividis)
    fig_amount.show()
    
    return year

In [ ]:
MAPINSDATA_Y= TX_count_amount(MAP_INS_DF,YR)

In [ ]:
Mapins_yr_dis(MAPINSDATA_Y,state)

In [ ]:
MAPINSDATA_Y_Q= TX_count_amount_Q(MAPINSDATA_Y,Quat)

In [ ]:
Mapins_yr_dis(MAPINSDATA_Y_Q,state) 

TOP DATA FUNCTIONS

INSURANCE

In [ ]:
def Topins_yr_pin(DF,state):

    year=DF[DF["States"]==state]
    year.reset_index(drop= True, inplace = True)

    col1, col2= st.columns(2)
    with col1:
        fig_count= px.bar(year, x="Quaters", y="Transaction_count", title=f"{year["Years"].min()} YEAR  TRANSACTION COUNT & PINCODES ",
                        color_discrete_sequence=px.colors.sequential.tempo_r, hover_name="Pincodes",width= 600)  

        st.plotly_chart(fig_count)
    with col2:

        fig_amount= px.bar(year, x="Quaters", y="Transaction_amount", title=f"{year["Years"].min()} YEAR  TRANSACTION AMOUNT & PINCODES ",
                        color_discrete_sequence=px.colors.sequential.Electric_r ,hover_name="Pincodes",width= 600)  

        st.plotly_chart(fig_amount)

In [ ]:
TOPINSDATA_Y= TX_count_amount(TOP_INS_DF,YR)

In [ ]:
TOPINSDATA_Y_PIN= Topins_yr_pin(TOPINSDATA_Y,state) 

In [ ]:
TOPINSDATA_Y_Q=TX_count_amount_Q(TOPINSDATA_Y,Quat)

USER

In [ ]:
def Topuser_yr(DF,YR):

    year=DF[DF["Years"]==YR]
    year.reset_index(drop= True, inplace = True)

    group_year=year.groupby(["States", "Quaters"])[[ "Registered_Users"]].sum()
    group_year.reset_index(inplace = True)

    fig_count= px.bar(group_year, x="States", y="Registered_Users",hover_name="States", title=f"{YR} REGISTERED USERS",color="Quaters",
                       color_discrete_sequence=px.colors.sequential.Emrld_r,height=800, width= 1100)

    st.plotly_chart(fig_count)

    return year

In [ ]:
TOPUSDATA_Y_S_Q=Topuser_yr(TOP_US_DF,YR)

In [ ]:
Topuser_yr_pin(TOPUSDATA_Y_S_Q,state)

TRANSACTION

In [ ]:
def Topuser_yr_pin(DF,state):
    year=DF[DF["States"]==state]
    year.reset_index(drop= True, inplace = True)

    fig_count= px.bar(year, x="Quaters", y="Registered_Users", title=f"{year["Years"].min()} YEAR {state.upper()} REGISTERED USERS & PINCODES",
                            color_continuous_scale=px.colors.sequential.tempo_r, width=1000, hover_name="Pincodes")  

    st.plotly_chart(fig_count)
    return year

In [ ]:
TOPTXDATA_Y= TX_count_amount(TOP_TX_DF,YR)

In [ ]:
Topins_yr_pin(TOPTXDATA_Y,state)

In [ ]:
TOPINSDATA_Y_Q= TX_count_amount_Q(TOPTXDATA_Y,Quat)

QUESTIONS

In [ ]:
#query1 Top 10 states With Highest Trasaction Amount',
myconnection = pymysql.connect(host = '127.0.0.1',user='root',passwd='root',database = "phonepe",port=3306)
cur = myconnection.cursor()

Query1 = '''select States, sum(Transaction_amount) as Transaction_Amount from agg_tx group by States order by Transaction_Amount desc limit 10;'''
cur.execute(Query1)
myconnection.commit()
F1=cur.fetchall()

DF1 = pd.DataFrame(F1,columns = ["States", "Transaction_Amount"])

fig_count= px.bar(DF1, x="States", y="Transaction_Amount", title="Highest Trasaction Amount",
                            color_continuous_scale=px.colors.sequential.deep_r, height=600, hover_name="States")  

fig_count.show()

In [ ]:
figpie_amount=px.pie(data_frame=DF1,names="States", values="Transaction_Amount",title="TRANSACTION AMOUNT",
                        hole= 0.1, width=600, )
figpie_amount.show() 

In [ ]:
#query2 Top 10 states With Lowest Trasaction Amount',
myconnection = pymysql.connect(host = '127.0.0.1',user='root',passwd='root',database = "phonepe",port=3306)
cur = myconnection.cursor()

Query2 = '''select States, sum(Transaction_amount) as Transaction_Amount from agg_tx group by States order by Transaction_Amount asc limit 10;'''
cur.execute(Query2)
myconnection.commit()
F2=cur.fetchall()

DF2 = pd.DataFrame(F2,columns = ["States", "Transaction_Amount"])

fig_count= px.bar(DF2, x="States", y="Transaction_Amount", title="Lowest Trasaction Amount",
                            color_continuous_scale=px.colors.sequential.deep_r, height=600, hover_name="States")  

fig_count.show()

In [ ]:
#query3 Top 10 mobile brands with Highest transaction count

myconnection = pymysql.connect(host = '127.0.0.1',user='root',passwd='root',database = "phonepe",port=3306)
cur = myconnection.cursor()

Query3 = '''select Brands, sum(Transaction_count) as Transaction_Count  from agg_us group by Brands order by Transaction_Count desc limit 10;'''
cur.execute(Query3)
myconnection.commit()
F3=cur.fetchall()

DF3 = pd.DataFrame(F3,columns = ["Brands", "Transaction_Count"])

fig_count= px.bar(DF3, x="Brands", y="Transaction_Count", title="Highest Trasaction count",
                            color_continuous_scale=px.colors.sequential.deep_r, height=600, hover_name="Brands")  

fig_count.show()

In [ ]:
#query4 Top 10 mobile brands with lowest transaction count

myconnection = pymysql.connect(host = '127.0.0.1',user='root',passwd='root',database = "phonepe",port=3306)
cur = myconnection.cursor()

Query4 = '''select Brands, sum(Transaction_count) as Transaction_Count  from agg_us group by Brands order by Transaction_Count asc limit 10; '''
cur.execute(Query4)
myconnection.commit()
F4=cur.fetchall()

DF4 = pd.DataFrame(F4,columns = ["Brands", "Transaction_Count"])

fig_count= px.bar(DF4, x="Brands", y="Transaction_Count", title="Lowest Trasaction count",
                            color_continuous_scale=px.colors.sequential.deep_r, height=600, hover_name="Brands")  

fig_count.show()

In [ ]:
#query5 Top 10 Registered-users based on District:

myconnection = pymysql.connect(host = '127.0.0.1',user='root',passwd='root',database = "phonepe",port=3306)
cur = myconnection.cursor()

Query5 = '''select States, Districts,  Registered_Users from map_user group by Districts order by Registered_Users desc limit 10;'''
cur.execute(Query5)
myconnection.commit()
F5=cur.fetchall()

DF5 = pd.DataFrame(F5,columns = ["States", "Districts", "Registered_Users"])

fig_count= px.line(DF5, x="Districts", y="Registered_Users", title="REGISTERED USERS", orientation="h",
                             height=600, hover_name="States", markers= True)  

fig_count.show()


In [ ]:
#Query 6 Top 10 Districts based on states with highest amount of transaction
myconnection = pymysql.connect(host = '127.0.0.1',user='root',passwd='root',database = "phonepe",port=3306)
cur = myconnection.cursor()

Query6 = '''select States, Districts, sum(Transaction_amount) as Transaction_Amount from map_tx group by Districts order by Transaction_Amount desc limit 10;'''
cur.execute(Query6)
myconnection.commit()
F6=cur.fetchall()

DF6 = pd.DataFrame(F6,columns = ["States", "Districts", "Transaction_Amount"])

fig_count= px.bar(DF6, x="Districts", y="Transaction_Amount", title="Transaction_Amount by District", orientation="v",
                             height=600, hover_name="States")  

fig_count.show()

In [ ]:
#Query 7 Top 10 Districts based on states with Lowest amount of transaction
myconnection = pymysql.connect(host = '127.0.0.1',user='root',passwd='root',database = "phonepe",port=3306)
cur = myconnection.cursor()

Query7 = '''select States, Districts, sum(Transaction_amount) as Transaction_Amount from map_tx group by Districts order by Transaction_Amount asc limit 10;'''
cur.execute(Query7)
myconnection.commit()
F7=cur.fetchall()

DF7 = pd.DataFrame(F7,columns = ["States", "Districts", "Transaction_Amount"])

fig_count= px.bar(DF7, x="Districts", y="Transaction_Amount", title="Transaction_Amount by District", orientation="v",
                             height=600, hover_name="States")  

fig_count.show()

In [ ]:
#query8 Top 10 States based on Districts with Highest amount of APP OPENS:

myconnection = pymysql.connect(host = '127.0.0.1',user='root',passwd='root',database = "phonepe",port=3306)
cur = myconnection.cursor()

Query8 = '''select States, Districts, sum(App_Opens) as APP_OPENS from map_user group by States order by APP_OPENS desc limit 10;'''
cur.execute(Query8)
myconnection.commit()
F8=cur.fetchall()

DF8 = pd.DataFrame(F8,columns = ["States", "Districts", "APP_OPENS"])

fig_count= px.line(DF8, x="States", y="APP_OPENS", title="APP OPENS", orientation="h",
                             height=600, hover_name="Districts", markers= True)  

fig_count.show()

In [ ]:
#query9 Top 10 States based on Districts with Lowest amount of APP OPENS:

myconnection = pymysql.connect(host = '127.0.0.1',user='root',passwd='root',database = "phonepe",port=3306)
cur = myconnection.cursor()

Query9 = '''select States, Districts, sum(App_Opens) as APP_OPENS from map_user group by States order by APP_OPENS asc limit 10;'''
cur.execute(Query9)
myconnection.commit()
F9=cur.fetchall()

DF9 = pd.DataFrame(F9,columns = ["States", "Districts", "APP_OPENS"])

fig_count= px.bar(DF9, x="States", y="APP_OPENS", title="APP OPENS",
                             height=600, hover_name="Districts")  

fig_count.show()

In [ ]:
#query10 plot a bar chart with all the pincodes based on transaction amount by each states:

myconnection = pymysql.connect(host = '127.0.0.1',user='root',passwd='root',database = "phonepe",port=3306)
cur = myconnection.cursor()

Query10 = '''select States, Pincodes, sum(Transaction_amount) as Transaction_Amount from top_ins group by Pincodes'''
cur.execute(Query10)
myconnection.commit()
F10=cur.fetchall()

DF10 = pd.DataFrame(F10,columns = ["States", "Pincodes", "Transaction_Amount"])

fig_count= px.bar(DF10, x="States", y="Transaction_Amount", title="Pincodes",
                             height=600, hover_name="Pincodes")  

fig_count.show()